# Data wrangling
Using hourly [FLUXNET](https://fluxnet.org/) data, compute monthly means and add IGBP information from the Biological, Ancillary, Disturbance, and Metadata (BADM) file as a column to the table. Output a csv file for use in other notebooks.<br><br>
This notebook uses the python package [pandas](https://pandas.pydata.org/docs/) to load, visualize, and resample data. 


**Input**: 
 - [FLUXNET 2015 SUBSET hourly data](https://fluxnet.org/data/fluxnet2015-dataset/subset-data-product/) for all tower sites
 - BADM file from FLUXNET 2015


**Output**: 
 - csv file for use in other notebooks

## Import notebook dependencies

In [1]:
import pandas as pd 
import numpy as np 
from glob import glob
import sys
import os

## Load FLUXNET 2015 files into notebook
Csv files from all tower sites in the FLUXNET 2015 dataset are stored in a directory in the repository. Here, we load in the files as DataFrame tables using pandas, and save the sitename (part of the filename for each file) as a column in the table.<br><br>A progress bar function is used in code development to show time remaining to run the code cell-- the cell loops through lots of data, and can take a while to run. 

In [2]:
#helper function used in code development
def progressBar(i, tot): 
    """Display a progress bar inside a for loop 
    Args: 
        i (int): iteration number
        tot (int): total number of iterations
    """
    j = (i + 1) / tot
    sys.stdout.write('\r [%-20s] %d%% complete' % ('='*int(20*j), 100*j))
    sys.stdout.flush()

In [3]:
# ----------------- load data ---------------------

#list of SUBSET filenames 
files = glob('data/FLUXNET2015/' + '*FLUXNET2015_SUBSET_HH*.csv')

#list for storing pandas dataframe objects
SUBSET_list = []

#loop through files in directory and add SUBSET hourly data to list
for i in range(len(files)):
    df = pd.read_csv(files[i])
    
    #add sitename to df 
    sitename = files[i].split('FLX_')[1].split('_FLUXNET2015')[0]
    df['SITE_ID'] = sitename
    
    #append to list
    SUBSET_list.append(df)
    
    #display progress bar
    progressBar(i, len(files))

 [====================] 100% complete

## Compute monthly averages
Resample the time series data, remove unneccessary columns, and add standard deviation of relative humidity to the dataset.

In [4]:
SUBSET_monthly = []
for i in range(len(SUBSET_list)): 
    df = SUBSET_list[i] 
    sitename = df["SITE_ID"][0]
    
    #remove unneccessary columns
    column_names = ["TIMESTAMP_START","SITE_ID","LE_F_MDS","VPD_F","TA_F","NETRAD","G_F_MDS", "PA_F", "RH"]
    if set(column_names).issubset(df.columns):
        #rename columns to include units
        df = df[column_names].rename(columns= {"LE_F_MDS":"LE (W/m^2)","VPD_F":"VPD (hPa)","TA_F":"TA (deg C)","NETRAD":"NETRAD (W/m^2)",
                                             "G_F_MDS":"G (W/m^2)","PA_F":"PA (kPa)"})
        #reformat time
        df["TIMESTAMP_START"] = pd.to_datetime(df["TIMESTAMP_START"], format='%Y%m%d%H%M')
        
        #convert VPD from hPa to kPa 
        df.insert(loc = 3, column = "VPD (kPa)", value = df["VPD (hPa)"]*0.1)
        df = df.drop(columns = "VPD (hPa)")
        
        #drop columns with missing data
        df = df.dropna() #drop columns with NaN
        missing_data = -9999.0 #missing data flag
        df = df[~df.eq(missing_data).any(1)].reset_index(drop = True) 

        #convert RH from percent to fraction
        df["RH"] = df["RH"]/100 
        
        #resample dataset to get monthly means
        df = df.set_index(["TIMESTAMP_START"])
        std_rh = df["RH"].resample('MS').std()
        df = df.resample('MS').mean()
             
        #append to list if dataframe contains data
        if len(df) > 0: 
            df["Std Dev RH"] = std_rh
            df.insert(loc = 0, column = "TIMESTAMP_START", value = df.index)
            df.insert(loc = 1, column = "SITE_ID", value = sitename)
            df = df.reset_index(drop = True)
            SUBSET_monthly.append(df)
        
    #display progress bar
    progressBar(i, len(SUBSET_list))

 [====================] 100% complete

## Load BADM file into notebook
This file contains data about each FLUXNET tower. We will add the IGBP ecotype corresponding to each tower to the dataset.

In [5]:
BADM = pd.read_csv('data/FLUXNET2015/FLX_AA-Flx_BIF_WW_20200501.csv', usecols = ['SITE_ID','VARIABLE','DATAVALUE'])
ecotype = BADM.loc[BADM['VARIABLE'] == 'IGBP'].reset_index(drop = True).drop(columns = 'VARIABLE')

## Merge BADM IGBP data with SUBSET HH
Add the BADM IGBP ecotype data as a column to the SUBSET hourly DataFrame. 

In [6]:
#combine dataframes
combined = pd.concat(SUBSET_monthly).merge(ecotype, on = 'SITE_ID', how = 'inner')

#rearrange order of IGBP column
combined.insert(loc = 2, column = "IGBP", value = combined["DATAVALUE"].values)
combined = combined.drop(columns = "DATAVALUE")

#rename columns
combined = combined.rename(columns= {"TIMESTAMP_START":"DATE", "SITE_ID":"FLUXNET-ID"})

#display part of dataframe 
display(combined[:10])

,DATE,FLUXNET-ID,IGBP,LE (W/m^2),VPD (kPa),TA (deg C),NETRAD (W/m^2),G (W/m^2),PA (kPa),RH,Std Dev RH
0,2011-01-01,US-CRT,CRO,5.317504,0.101241,-5.489418,9.618696,-4.961629,99.831905,0.757487,0.102424
1,2011-02-01,US-CRT,CRO,11.828405,0.145528,-2.971717,20.088240,-0.771094,99.859839,0.737630,0.134346
2,2011-03-01,US-CRT,CRO,34.717653,0.197390,2.153819,92.061066,11.126964,100.084974,0.747895,0.144803
3,2011-04-01,US-CRT,CRO,45.258565,0.307019,7.368077,96.613316,11.045620,99.294018,0.737826,0.150827
4,2011-05-01,US-CRT,CRO,82.249253,0.444950,15.870246,143.585133,13.552872,99.448677,0.794998,0.157474
5,2011-06-01,US-CRT,CRO,53.869295,0.992942,22.068460,172.166276,8.117042,99.467544,0.676487,0.183932
6,2011-07-01,US-CRT,CRO,125.305980,1.061575,25.869129,183.696062,6.692552,99.519419,0.717682,0.184548
7,2011-08-01,US-CRT,CRO,137.802859,0.581209,21.853141,161.626838,-0.528036,99.361281,0.806610,0.152929
8,2011-09-01,US-CRT,CRO,73.312711,0.347930,17.358452,100.017480,-3.180038,99.601412,0.853520,0.136794
9,2011-10-01,US-CRT,CRO,36.766119,0.394087,11.548182,73.515675,-5.649803,99.628152,0.763188,0.181707


## Save DataFrame as csv to local drive 
This csv file will be used in other notebooks for data analysis, and is available as "FLUXNET2015_monthly.csv" in the GitHub repository for this project. 

In [7]:
combined.to_csv('data/FLUXNET2015_monthly.csv', index = False)